In [ ]:
%matplotlib ipympl

from datetime import datetime
import glob
import os

import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
import numpy as np
import xarray as xr

import ls2d

from microhhpy.spatial import Domain, plot_domains
from microhhpy.land import create_land_surface_input
from microhhpy.land import get_ifs_soil_lut, get_ifs_vegetation_lut, get_ifs_vegetation_cmap
from microhhpy.land import read_hihydrosoil_subtop
from microhhpy.real import create_sst_from_regular_latlon
from microhhpy.interp import extrapolate_onto_mask, interp_rect_to_curv_latlon_2d

TF = np.float64

## Realistic heterogeneous land-use
For realistic cases, we provide methods to create land-surface input from:
- [Corine, Europe, 100m resolution](https://land.copernicus.eu/en/products/corine-land-cover)
- [LCC, Global, 100 m resolution](https://land.copernicus.eu/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2019-globe)
- [LCC, Global 10 m resolution](https://land.copernicus.eu/en/products/global-dynamic-land-cover/land-cover-2020-raster-10-m-global-annual) (Not yet, TODO!)

In [ ]:
corine_tiff = '/home/scratch1/bart/Corine/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif'
lcc_tiff = '/home/scratch1/bart/LCC/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif'
hihydrosoil_tiffs = '/home/scratch1/bart/HiHydroSoil_v2.0'

In [ ]:
"""
Setup domain / projection.
"""
lon = 5.3
lat = 53.1
proj_str = f'+proj=lcc +lat_1={lat-1} +lat_2={lat+1} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

dom = Domain(
    xsize = 204800,
    ysize = 204800,
    itot = 1024,
    jtot = 1024,
    lon = lon,
    lat = lat,
    anchor = 'center',
    proj_str = proj_str)

plot_domains([dom], use_projection=True)

z_soil = np.array([-0.035, -0.175, -0.64 , -1.945])[::-1]

lon = dom.proj.lon
lat = dom.proj.lat

In [ ]:
lu_corine = create_land_surface_input(
    lon,
    lat,
    z_soil,
    land_use_source='corine_100m',
    land_use_tiff=corine_tiff,
    save_binaries=True,
    output_dir='.',
    save_netcdf=True,
    netcdf_file='lsm_input_corine.nc',
)

lu_lcc = create_land_surface_input(
    lon,
    lat,
    z_soil,
    land_use_source='lcc_100m',
    land_use_tiff=lcc_tiff,
    save_binaries=True,
    output_dir='.',
    save_netcdf=True,
    netcdf_file='lsm_input_lcc.nc',
)

In [ ]:
veg_lut = get_ifs_vegetation_lut()
names = veg_lut['name']
ifs_cmap = get_ifs_vegetation_cmap()

plt.figure(figsize=(10, 5), layout='constrained')

ax=plt.subplot(121)
plt.title('Corine 100 m')
plt.pcolormesh(lu_corine.lon, lu_corine.lat, lu_corine.ds.index_veg, vmin=0, vmax=20, cmap=ifs_cmap)

plt.subplot(122, sharex=ax, sharey=ax)
plt.title('LCC 100 m')
plt.pcolormesh(lu_lcc.lon, lu_lcc.lat, lu_lcc.ds.index_veg, vmin=0, vmax=20, cmap=ifs_cmap)

cbar = plt.colorbar(shrink=0.8)
cbar.set_ticks(np.arange(21))
cbar.set_ticklabels([veg_type.replace('_', ' ').title() for veg_type in names])
cbar.ax.tick_params(labelsize=8)

## Sea Surface

Initializing sea surface temperatures (SSTs) can be challenging, especially when spatial interpolation are needed between sea and land grid points in the host model. To overcome this, we offer a method that first extrapolates SSTs over land areas before performing the interpolation. 

In [ ]:
settings = {
    'start_date'  : datetime(year=2022, month=4, day=1, hour=8),
    'end_date'    : datetime(year=2022, month=4, day=1, hour=20),
    'central_lon' : 4.8,
    'central_lat' : 53,
    'area_size'   : 5,
    'case_name'   : 'slocs_rf',
    'era5_path'   : '/home/scratch1/bart/LS2D_ERA5/',
    'era5_expver' : 1,
    'cdsapirc'    : '/home/bart/.cdsapirc_ads'
    }

era5 = ls2d.Read_era5(settings)
era5.calculate_forcings(n_av=3, method='2nd')

In [ ]:
sst_era = era5.sst[0]

sst_les = create_sst_from_regular_latlon(
    era5.sst[0],
    era5.lons,
    era5.lats,
    lon,
    lat,
    float_type=TF)

# We don't know water temperatures over land, and the extrapolated
# SSTs are of course not a very accurate estimation...
# TODO: get inland water mask from Corine/LCC and let user define water/lake temperatures?
land_mask = lu_lcc.water_mask < 1
sst_les[sst_les < 273.15] = 273.15 + 15

# Just for plotting...
sst_les = np.ma.masked_array(sst_les, mask=land_mask)

plt.figure(figsize=(8, 3.5), layout='constrained')

plt.subplot(121)
plt.title('ERA5 SST')
plt.pcolormesh(era5.lons, era5.lats, era5.sst[0], vmin=278, vmax=283)
plt.colorbar()
plt.xlim(lon.min(), lon.max())
plt.ylim(lat.min(), lat.max())

plt.subplot(122)
plt.title('LES SST')
plt.pcolormesh(lon, lat, sst_les, vmin=278, vmax=283)
plt.colorbar()
plt.xlim(lon.min(), lon.max())
plt.ylim(lat.min(), lat.max())

## Soil moisture: TODO

Soil moisture is equally challenging, as many ERA5 values near the coast result from interpolations between sea and land grid points, leading to artificially low values. In addition, the soil dataset also does not have values over sea..

In [ ]:
#lon = dom.proj.lon
#lat = dom.proj.lat
#
## HiHydroSoil dataset (250 m resolution).
#ds_hihy = read_hihydrosoil_subtop(hihydrosoil_tiffs, lon.min(), lon.max(), lat.min(), lat.max())
#
## Extrapolate all known values a few grid points onto sea to allow interpolations near the coast.
#for var in ds_hihy:
#    ds_hihy[var] = (ds_hihy[var].dims, extrapolate_onto_mask(ds_hihy[var].values, ~np.isnan(ds_hihy[var]), max_distance=5))
#
## Interpolate dataset onto LES grid (NN).
#latlon_out = np.column_stack([lat.ravel(), lon.ravel()])
#
#data_vars = {}
#for name, da in ds_hihy.data_vars.items():
#    ip = RegularGridInterpolator((ds_hihy.y, ds_hihy.x), da.values, method='nearest', bounds_error=True)
#    fld_out = ip(latlon_out).reshape(lat.shape)
#    data_vars[name] = (('y', 'x'), fld_out)
#
#ds_hihy_les = xr.Dataset(
#    data_vars=data_vars,
#    coords={'lat': (('y','x'), lat),
#            'lon': (('y','x'), lon)}
#)

In [ ]:
## Soil lookup table with hydraulic properties.
#vg_ds = get_ifs_soil_lut()

In [ ]:
## Process ERA5 data: from absolute to relative soil moisture, which is something we can spatially interpolate.
#theta_soil = era5.theta_soil[0]
#soil_index = era5.soil_type[0,:,:] - 1      # FØRTRAN to C.
#
#theta_wp = vg_ds.theta_wp.values[soil_index]
#theta_fc = vg_ds.theta_fc.values[soil_index]
#theta_rel = (theta_soil - theta_wp) / (theta_fc - theta_wp)
#
## Mask grid points which are partially sea. In ERA5, these have been interpolated between land and sea,
## where sea has a soil moisture content of zero, so soil moisture is artificially low.
#land_mask = era5.sst[0].mask == 1
#
## Extrapolate all known values a few grid points onto sea to allow interpolations near the coast.
#theta_rel_ext = np.empty_like(theta_rel)
#
#for k in range(theta_rel.shape[0]):
#    theta_rel_ext[k] = extrapolate_onto_mask(theta_rel[k], land_mask, max_distance=2)
#
#theta_rel_les = np.zeros((4, lon.shape[0], lon.shape[1]), dtype=TF)
#for k in range(4):
#    theta_rel_les[k] = interp_rect_to_curv_latlon_2d(theta_rel_ext[k,:,:], era5.lons, era5.lats, lon, lat, float_type=TF, method='linear')

In [ ]:
# Translate interpolated relative soil moisture content back to absolute using HiHydroSoil properties interpolated to LES grid.


In [ ]:
# Cleanup!
files = glob.glob('*00*') + glob.glob('*.nc')
for f in files:
    os.remove(f)